In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Age option 2 APOKASC-2 Pinsonneault et al. 2018
#Reading in the table, making sure all the tables have a column named Age in Gyr
# and that every star in the table has an Age
apokasc2raw = Table.read("Pinsonneault2018.txt", format="ascii.cds")
apokasc2raw['Age']=(10**np.array(apokasc2raw['LogAge'])/1000) #Age was in log(Myr) so needs converting
hasagea2=np.where((apokasc2raw['Age']==apokasc2raw['Age']) & (apokasc2raw['Age']>0.1))
agedata=apokasc2raw[hasagea2]

In [2]:
from tensorflow import keras

In [3]:
filename='astraAllStarASPCAP-0.6.0.fits'
tb = fits.open(filename)
header=tb[2].header
data = tb[2].data 

mask_gaia = (data['zgr_plx']>0)
data_masked=data[mask_gaia]

intersect, ind_a, ind_b = np.intersect1d(data_masked['sdss4_apogee_id'],agedata['2MASS'], return_indices=True) 

fullx = np.dstack([data_masked['teff'][ind_a],data_masked['logg'][ind_a], data_masked['m_h_atm'][ind_a],
                   data_masked['alpha_m_atm'][ind_a], data_masked['c_h'][ind_a], data_masked['n_h'][ind_a]])[0]

fully = np.dstack([agedata['Age'][ind_b]])[0] #for Pinsonneault 2018

#remove non-finite entries!
mask = np.all(np.isfinite(fullx), axis=1) & np.all(np.isfinite(fully), axis=1)
fullx, fully = fullx[mask], fully[mask]

scaling_x = np.median(fullx, axis=0)
scaling_y = np.median(fully, axis=0)

fullx, fully = fullx/scaling_x, fully/scaling_y

In [6]:
neurons_per_layer=50
layers=6
iterations=50

In [8]:

inputs = keras.Input(shape=(6,))

layer1 = keras.layers.Dense(neurons_per_layer, activation='relu')(inputs)
layer2 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer1)
layer3 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer2)
layer4 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer3)
layer5 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer4)
layer6 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer5)

outputs = keras.layers.Dense(1)(layer6)

model = keras.Model(inputs=inputs, outputs=outputs, name='test')

model.summary()

Model: "test"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 6)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 50)                  │             350 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,151 (51.37 KB)

 Trainable params: 13,151 (51.37 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

tenpercent=len(agedata['Age'][ind_b])//10 

trainbin=slice(0,-1*tenpercent-1)
testing=slice(-1*tenpercent,-1)

x_train, y_train = fullx[trainbin], fully[trainbin]
x_test, y_test = fullx[testing], fully[testing]

In [12]:
model.fit(x_train, y_train, epochs=iterations, validation_split=0.05, batch_size=300)

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 5.5876e-04 - loss: 0.7780 - val_accuracy: 0.0000e+00 - val_loss: 0.4303
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0013 - loss: 0.4087 - val_accuracy: 0.0000e+00 - val_loss: 0.3155
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0013 - loss: 0.3286 - val_accuracy: 0.0000e+00 - val_loss: 0.2707
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0013 - loss: 0.2857 - val_accuracy: 0.0000e+00 - val_loss: 0.2462
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0013 - loss: 0.2615 - val_accuracy: 0.0000e+00 - val_loss: 0.2382
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0013 - loss: 0.2640 - val_accuracy: 0.0000e+00 - val_loss: 0.2423
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0013 - loss: 0.2548 - val_accuracy: 0.0000e+00 - val_loss: 0.2405
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0013 - loss: 0.2539 -

In [13]:
predictions = model.predict(x_test)
print(len(predictions))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
651


In [15]:
neurons_per_layer_model2=500
layers_model2=14
iterations_model2=200

In [16]:
inputs2 = keras.Input(shape=(6,))

layer1model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(inputs2)
layer2model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer1model2)
layer3model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer2model2)
layer4model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer3model2)
layer5model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer4model2)
layer6model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer5model2)
layer7model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer6model2)
layer8model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer7model2)
layer9model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer8model2)
layer10model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer9model2)
layer11model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer10model2)
layer12model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer11model2)
layer13model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer12model2)
layer14model2 = keras.layers.Dense(neurons_per_layer_model2, activation='relu')(layer13model2)

outputs2 = keras.layers.Dense(1)(layer14model2)

model2 = keras.Model(inputs=inputs2, outputs=outputs2, name='second_model')

model2.summary()

Model: "second_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 6)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 500)                 │           3,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 1)                   │             501 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,260,501 (12.44 MB)

 Trainable params: 3,260,501 (12.44 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model2.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model2.fit(x_train, y_train, epochs=iterations_model2, validation_split=0.05, batch_size=300)

Epoch 1/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - accuracy: 5.5876e-04 - loss: 1.4035 - val_accuracy: 0.0000e+00 - val_loss: 0.4178
Epoch 2/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.0013 - loss: 0.4020 - val_accuracy: 0.0000e+00 - val_loss: 0.3137
Epoch 3/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.0013 - loss: 0.3209 - val_accuracy: 0.0000e+00 - val_loss: 0.2401
Epoch 4/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.0013 - loss: 0.2714 - val_accuracy: 0.0000e+00 - val_loss: 0.2376
Epoch 5/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.0013 - loss: 0.2563 - val_accuracy: 0.0000e+00 - val_loss: 0.2302
Epoch 6/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.0013 - loss: 0.2465 - val_accuracy: 0.0000e+00 - val_loss: 0.2219
Epoch 7/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.0013 - loss: 0.2341 - val_accuracy: 0.0000e+00 - val_loss: 0.2290
Epoch 8/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.0013 -

In [18]:
predictions2 = model2.predict(x_test)
print(len(predictions2))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
651


In [20]:
metric=0.3 #is the accuracy better than 30%?
goodfit=np.where(((1-metric) < predictions/y_test) & ((1+metric) > predictions/y_test)) 
badfit=np.where(((1-metric) > predictions/y_test) | ((1+metric) < predictions/y_test))

print ('With ', neurons_per_layer, 'neurons per layer, ', layers, 'layers, and ', iterations, 'iterations')
print ('using the training set', trainbin)
print (len(goodfit[0])/len(y_test)*100, 'percent of the ages are good')
print (len(badfit[0])/len(y_test)*100, 'percent of the ages are bad')

With  50 neurons per layer,  6 layers, and  50 iterations
using the training set slice(0, -653, None)
55.4531490015361 percent of the ages are good
44.5468509984639 percent of the ages are bad


In [21]:
goodfit2=np.where(((1-metric) < predictions2/y_test) & ((1+metric) > predictions2/y_test)) 
badfit2=np.where(((1-metric) > predictions2/y_test) | ((1+metric) < predictions2/y_test))

print ('With ', neurons_per_layer_model2, 'neurons per layer, ', layers_model2, 'layers, and ', iterations_model2, 'iterations')
print ('using the training set', trainbin)
print (len(goodfit2[0])/len(y_test)*100, 'percent of the ages are good')
print (len(badfit2[0])/len(y_test)*100, 'percent of the ages are bad')

With  500 neurons per layer,  14 layers, and  200 iterations
using the training set slice(0, -653, None)
59.754224270353305 percent of the ages are good
40.2457757296467 percent of the ages are bad


In [32]:
difference = abs(predictions-predictions2)
print(difference)

from astropy.table import QTable
import astropy.units as u

totaldata = Table([predictions, predictions2, difference, np.mean(difference)], names=('Model 1 Predictions (Gyr)','Model 2 Predictions (Gyr)','Difference (Gyr)','Average Difference'), meta={'name': 'first table'})
print(totaldata)

print("The average difference is",np.mean(difference),"Gyr")

[[7.91299939e-02]
 [1.26242638e-04]
 [8.19668770e-02]
 [4.81435657e-03]
 [8.65528882e-02]
 [3.99515927e-02]
 [1.41975999e-01]
 [2.12955475e-03]
 [1.59482062e-02]
 [1.88561380e-02]
 [9.26231146e-02]
 [1.67478919e-02]
 [3.34101915e-03]
 [2.47910976e-01]
 [2.41207361e-01]
 [8.96950066e-02]
 [6.61913157e-02]
 [5.55982888e-01]
 [9.75933671e-02]
 [3.36896777e-01]
 [3.46436977e-01]
 [1.22151852e-01]
 [1.04283094e-02]
 [3.09345126e-02]
 [2.04690695e-01]
 [8.49452019e-02]
 [2.53187418e-01]
 [2.25432813e-02]
 [2.70873785e-01]
 [3.80046368e-02]
 [4.95135188e-02]
 [3.83322984e-02]
 [4.70693409e-02]
 [6.74554706e-02]
 [1.69928074e-02]
 [4.78577018e-02]
 [9.74488258e-03]
 [4.29600179e-02]
 [2.88352460e-01]
 [1.11434042e-01]
 [3.13480377e-01]
 [2.51643300e-01]
 [2.67831266e-01]
 [2.63593376e-01]
 [8.04409981e-02]
 [3.54425967e-01]
 [4.56491709e-02]
 [1.66430950e-01]
 [9.51451957e-02]
 [1.27352595e-01]
 [1.16070271e-01]
 [1.51743710e-01]
 [7.75891781e-01]
 [1.62318647e-02]
 [6.60456121e-02]
 [7.487881

TypeError: len() of unsized object

In [30]:
totaldata.write("APOKASC2 trained on 2 different neural models.ecsv")